In [1]:
import sqlite3
import csv
import re

import pandas as pd # to load column names and their data types
import random, string # to generate random table name if necessary

In [4]:
csv_file = "processed_data/processed_phish.csv"

In [6]:
df_from_csv = pd.read_csv(csv_file)
df_from_csv.head()

,combined_text,label
0,hpl nom may see attached file hplno xl hplno xl,0
1,nom actual vols th forwarded sabrae zajac hou ...,0
2,enron actuals march april estimated actuals ma...,0
3,hpl nom may see attached file hplno xl hplno xl,0
4,hpl nom june see attached file hplno xl hplno xl,0


In [7]:
df_from_csv.dtypes.unique()

array([dtype('O'), dtype('int64')], dtype=object)

In [8]:
def get_table_name(csv_file):
    '''
    Create a table name from CSV file name and convert it to be table name
    allowed by slite3 documentation.
    '''
    # when in CSV file name there are letters too
    regex = re.compile('[^a-z]')
    table_name = csv_file.split("/")[-1].split(".")[0]
    table_name = regex.sub('', table_name)

    # when in CSV file name there aren't any letters
    if table_name == '':
        for i in range(10):
            table_name += random.choice(string.ascii_lowercase)
    return table_name

generated_table_name = get_table_name(csv_file)
generated_table_name

'processedphish'

In [12]:
def create_table(df_dataset, table_name):
    cols_with_sql_types = []
    for col_name, col_type in df_dataset.dtypes.items():
        if col_type == "int64":
            cols_with_sql_types.append('"' + col_name + '"' + ' ' + 'INTEGER')
        elif col_type == "float64":
            cols_with_sql_types.append('"' + col_name + '"' + ' ' + 'REAL')
        else:
            cols_with_sql_types.append('"' + col_name + '"' + ' ' + 'TEXT')
        
    final = str(cols_with_sql_types).replace("'", "").replace(']', '').replace('[', '')
    return f'CREATE TABLE "{table_name}" ({final})'

In [13]:
def drop_table_if_exists(table_name):
    return f'DROP TABLE IF EXISTS {table_name}'

In [14]:
conn = sqlite3.connect(f'{generated_table_name}.sqlite')
cur = conn.cursor()
cur.execute(f'{drop_table_if_exists(generated_table_name)}')
cur.execute(f'{create_table(df_from_csv, generated_table_name)}')

In [15]:
def insert_into_values(df_dataset, table_name):
    '''
    The function returns SQL statement "INSERT INTO" with needed table name and values.
    '''
    numb_of_columns = len(df_dataset.columns)
    values = str(['?' for i in range(numb_of_columns)]).replace("'", "").replace(']', '').replace('[', '')
    return f'INSERT INTO "{table_name}" VALUES ({values})'

In [16]:
def convert_to_str(df_dataset):
    '''
    The function converts problematic dtypes to strings.
    '''
    for i in df_dataset.select_dtypes(include=['datetime', 'timedelta']):
        df_dataset[i] = df_dataset[i].astype(str)

    return df_dataset

In [17]:
def executemany(df_dataset, table_name):
    with sqlite3.connect(f'{table_name}.sqlite'):
        conn = sqlite3.connect(f'{table_name}.sqlite')
        cur = conn.cursor()

        values = convert_to_str(df_dataset).values.tolist()

        cur.executemany(f"{insert_into_values(df_dataset, table_name)}", values)
        conn.commit()

executemany(df_from_csv, generated_table_name)

In [18]:
df_from_sql = pd.read_sql_query(f'select * from {generated_table_name}', con = conn)
df_from_sql

,combined_text,label
0,hpl nom may see attached file hplno xl hplno xl,0
1,nom actual vols th forwarded sabrae zajac hou ...,0
2,enron actuals march april estimated actuals ma...,0
3,hpl nom may see attached file hplno xl hplno xl,0
4,hpl nom june see attached file hplno xl hplno xl,0
...,...,...
82478,info advantageapartmentscom infoadvantageapart...,1
82479,monkeyorg helpdeskmonkeyorg monkeyorg hi josep...,1
82480,help center infohelpcentercozainfohelpcenterco...,1
82481,metamask infosofamekarcom verify metamask wall...,1
